In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
#######################################################
############### If using Google Drive #################
#######################################################

# Please note that if this cell does not run go to where the shared folder is on Google Drive, and
# right-click on the shared folder, and select Add shortcut to Drive. Then try execute the cell again.

# Mount Google Drive where datasets are located
from google.colab import drive
drive.mount('/content/gdrive')

# Change the current working directory
%cd /content/gdrive/MyDrive/DS\ Capstone\ Project/REPORT_CODE_STRUCTURED/

# Define data path
data_path = 'data/original/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/107U69c8Nh3fH_vc0lG1KAZi0i92sHvcb/DS Capstone Project/REPORT_CODE_STRUCTURED


In [ ]:
#######################################################
############## If using local Computer ################
#######################################################

# Define data path
data_path = '../data/original/'

In [ ]:
# Load each dataset into memory
df_fc_qld = pd.read_csv(f'{data_path}forecastdemand_qld.csv')
df_fc_sa = pd.read_csv(f'{data_path}forecastdemand_sa.csv')
df_fc_vic = pd.read_csv(f'{data_path}forecastdemand_vic.csv')
df_fc_nsw = pd.read_csv(f'{data_path}forecastdemand_nsw.csv')

df_temp_qld = pd.read_csv(f'{data_path}temprature_qld.csv')
df_temp_sa = pd.read_csv(f'{data_path}temprature_sa.csv')
df_temp_vic = pd.read_csv(f'{data_path}temprature_vic.csv')
df_temp_nsw = pd.read_csv(f'{data_path}temperature_nsw.csv')

df_td_qld = pd.read_csv(f'{data_path}totaldemand_qld.csv')
df_td_sa = pd.read_csv(f'{data_path}totaldemand_sa.csv')
df_td_vic = pd.read_csv(f'{data_path}totaldemand_vic.csv')
df_td_nsw = pd.read_csv(f'{data_path}totaldemand_nsw.csv')

In [ ]:
# Display more than the standard 60 rows per pandas dataframe
pd.set_option('display.max_rows', 500)

<u style="font-size:28px;color:rgba(25,167,255,0.8)"><b>What the columns mean</b></u>

<u style="font-size:16px">Forecast Demand</u>

1. **PREDISPATCHSEQNO** - Unique identifier of predispatch run (YYYYMMDDPP). In energy generation, “dispatch” refers to process of sending out energy to the power grid to meet energy demand. “Predispatch” then is an estimated forecast of this amount.

2. **REGIONID** - Region Identifier (i.e. NSW1)

3. **PERIODID** - Period count, starting from 1 for each predispatch run.

4. **FORECASTDEMAND** - Forecast demand (MW) in half-hourly increments in NSW

5. **LASTCHANGED** - Date time interval of each update of the observation (dd/mm/yyyy hh:mm)

6. **DATETIME** - Date time interval of each observation (dd/mm/yyyy hh:mm)

For example, given the below: Forecast demand for 1/1/2010 0:00, measured on 30/12/2009 12:31:49pm was 7832.04 MW

<u style="font-size:16px">Air temperature</u>

1. **LOCATION** - Location of a weather station (i.e. Bankstown weather station)

2. **DATETIME** - Date time interval of each observation (dd/mm/yyyy hh:mm)

3. **TEMPERATURE** -  Air temperature (degrees celsius)

<u style="font-size:16px">Total Electricity Demand</u>

1. **DATETIME** - Date and time interval of each observation in the format (dd/mm/yyyy hh:mm)

2. **TOTALDEMAND** - Total demand (MW)

3. **REGIONID** - Region Identifier (i.e. NSW1)

<div style="background-color:rgba(255,40,45,0.7);font-size:24px;text-align:center">Datasets</div>

<div style="background-color:rgba(10,219,0,0.5);font-size:24px;padding-left:20px">Total Demand</div>

In [ ]:
# Merge all Total Demand datasets together into one
df_td = pd.concat([df_td_qld, df_td_sa, df_td_vic, df_td_nsw]).reset_index(drop=True)

In [ ]:
df_td.head(5)

,DATETIME,TOTALDEMAND,REGIONID
0,2010-01-01 00:00:00,5561.21,QLD1
1,2010-01-01 00:30:00,5422.25,QLD1
2,2010-01-01 01:00:00,5315.98,QLD1
3,2010-01-01 01:30:00,5186.70,QLD1
4,2010-01-01 02:00:00,5050.83,QLD1


In [ ]:
df_td.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786051 entries, 0 to 786050
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATETIME     786051 non-null  object 
 1   TOTALDEMAND  786051 non-null  float64
 2   REGIONID     786051 non-null  object 
dtypes: float64(1), object(2)
memory usage: 18.0+ MB


In [ ]:
df_td.describe()

,TOTALDEMAND
count,786051.000000
mean,5194.140855
std,2606.719181
min,21.890000
25%,2983.865000
50%,5601.170000
75%,6921.285000
max,14579.860000


In [ ]:
# Checking missing values 
def missing_values(df):
    global df_mv_cols
    mv = df.isnull().sum().sort_values(ascending=False)
    percentage = round(mv / len(df) * 100, 2)   # rounded 2 dp
    df_mv = pd.concat([mv, percentage], axis=1, keys=['Total', 'Percentage'])
    if sum(mv) > 0:
        return df_mv.loc[df_mv['Total'] > 0]
    else:
        return "No Missing Values"
    
missing_values(df_td)

'No Missing Values'

### REGIONID Column

In [ ]:
# Let's get the unique values
df_td['REGIONID'].unique()

array(['QLD1', 'SA1', 'VIC1', 'NSW1'], dtype=object)

In [ ]:
# Removing the 1 on the end of each value
df_td.replace({'QLD1': 'QLD', 'SA1': 'SA', 'VIC1': 'VIC', 'NSW1': 'NSW'}, inplace=True)

In [ ]:
# Check to make sure there correct
unique_regionids = list(df_td['REGIONID'].unique())
unique_regionids

['QLD', 'SA', 'VIC', 'NSW']

### DATETIME Column

We want to see if there is any missing values between the lowest and highest DATETIMES for each REGIONID. 
But before that, we need to make sure the data is in consistent format

In [ ]:
### For each REGIONID, let's make sure DATETIMES are in 'yyyy-mm-dd hh:mm:ss' format
for i in df_td.groupby('REGIONID'):
    print(i)

('NSW',                DATETIME  TOTALDEMAND REGIONID
589538    1/1/2010 0:00      8038.00      NSW
589539    1/1/2010 0:30      7809.31      NSW
589540    1/1/2010 1:00      7483.69      NSW
589541    1/1/2010 1:30      7117.23      NSW
589542    1/1/2010 2:00      6812.03      NSW
...                 ...          ...      ...
786046  17/3/2021 22:00      7419.77      NSW
786047  17/3/2021 22:30      7417.91      NSW
786048  17/3/2021 23:00      7287.32      NSW
786049  17/3/2021 23:30      7172.39      NSW
786050   18/3/2021 0:00      7094.51      NSW

[196513 rows x 3 columns])
('QLD',                    DATETIME  TOTALDEMAND REGIONID
0       2010-01-01 00:00:00      5561.21      QLD
1       2010-01-01 00:30:00      5422.25      QLD
2       2010-01-01 01:00:00      5315.98      QLD
3       2010-01-01 01:30:00      5186.70      QLD
4       2010-01-01 02:00:00      5050.83      QLD
...                     ...          ...      ...
196508  2021-03-17 22:00:00      6443.62      QLD
1965

In [ ]:
# Only NSW is not in 'yyyy-mm-dd hh:mm:ss' format, so let's fix it
for regionid in unique_regionids:
    if regionid == 'NSW':
        df_td.loc[df_td['REGIONID'] == regionid, 'DATETIME'] = pd.to_datetime(df_td.loc[df_td['REGIONID'] == regionid, 'DATETIME'], format='%d/%m/%Y %H:%M')
    else:
        df_td.loc[df_td['REGIONID'] == regionid, 'DATETIME'] = pd.to_datetime(df_td.loc[df_td['REGIONID'] == regionid, 'DATETIME'], format='%Y-%m-%d %H:%M:%S')
        
# Now converting 'DATETIME' column to datetime64[ns] - the above won't do it
df_td['DATETIME'] = pd.to_datetime(df_td['DATETIME'])

In [ ]:
# Let's check for any duplicated DATETIMES 
def check_duplicate_datetimes(df, REGIONIDS=unique_regionids, col='REGIONID'):
    '''
    Checks to see whether there are any missing DATETIMES (half-hourly) from the minimum DATETIME in df to the maximum DATETIME in df
    '''
    for regionid in REGIONIDS:
        region_dts = df.loc[df[col] == regionid, 'DATETIME']
        
        # Get length of region datetimes
        length_region_dts = len(region_dts)
        
        # Get uniques of region datetimes
        length_unique_region_dts = len(region_dts)
        
        # Check to see if any missing datetimes
        ### Get min and max DATETIMES
        minMaxDatetimes = region_dts.astype('str').agg(['min', 'max'])
        min_, max_ = minMaxDatetimes['min'], minMaxDatetimes['max']
        
        ### Get all DATETIME combinations (half-hourly) from min_ to max_ 
        allDatetimeCombs = pd.date_range(start=min_, end=max_, freq='30T')
        
        ### Get length of all datetime combinations
        length_all_combs = len(allDatetimeCombs)

        print(f'{regionid}:\nColumn = {length_region_dts}   |   Column Uniques = {length_unique_region_dts}   |   All Combinations = {length_all_combs}\n')

regionid_duplicate_dts = check_duplicate_datetimes(df_td)

QLD:
Column = 196513   |   Column Uniques = 196513   |   All Combinations = 196513

SA:
Column = 196512   |   Column Uniques = 196512   |   All Combinations = 196512

VIC:
Column = 196513   |   Column Uniques = 196513   |   All Combinations = 196513

NSW:
Column = 196513   |   Column Uniques = 196513   |   All Combinations = 196513



In [ ]:
# Returns a dictionary mapping each state to a list of missing DATETIMES if there is any. Not really needed if all rows have same value as in previous cell
def check_missing_datetimes(df, REGIONIDS=unique_regionids, col='REGIONID'):
    '''
    Checks to see whether there are any missing DATETIMES (half-hourly) from the minimum DATETIME in df to the maximum DATETIME in df
    '''
    region_missing_dts = {}
    for regionid in REGIONIDS:
        region_dts = df.loc[df[col] == regionid, 'DATETIME']
        
        # Get min and max DATETIMES
        minMaxDatetimes = region_dts.astype('str').agg(['min', 'max'])
        min_, max_ = minMaxDatetimes['min'], minMaxDatetimes['max']
        
        # Get all DATETIME combinations (half-hourly) from min_ to max_ 
        allDatetimeCombs = pd.date_range(start=min_, end=max_, freq='30T')

        # Find any possible missing DATETIMES in df
        missing = list(set(allDatetimeCombs) - set(region_dts))
        if len(missing) == 0:
            print(f'REGIONID {regionid} has no missing DATETIME between {min_} and {max_}')
        else:
            region_missing_dts[regionid] = missing
            print(f'REGIONID {regionid} has missing DATETIMES between {min_} and {max_}')
    
    if len(region_missing_dts) != 0:
        return region_missing_dts

regionid_missing_dts = check_missing_datetimes(df_td)
regionid_missing_dts

REGIONID QLD has no missing DATETIME between 2010-01-01 00:00:00 and 2021-03-18 00:00:00
REGIONID SA has no missing DATETIME between 2010-01-01 00:30:00 and 2021-03-18 00:00:00
REGIONID VIC has no missing DATETIME between 2010-01-01 00:00:00 and 2021-03-18 00:00:00
REGIONID NSW has no missing DATETIME between 2010-01-01 00:00:00 and 2021-03-18 00:00:00


### TOTALDEMAND Column

See descriptive statistics above since this column is numeric

## Visualisation

df_tdNow the data is cleaned into a better format, let's now visualise the data

In [ ]:
df_td_vis = df_td.copy()

In [ ]:
# First, we'll introduce some new variables based on the DATETIME column to make visualisation easier
# Let's create some new features derived from DATETIME
def create_new_features_td(df):
    df["HOUR"] = df["DATETIME"].dt.hour
    df["ISO_DAYOFWEEK"] = df["DATETIME"].dt.isocalendar().day.astype('int') # need to change from uint to int
    df["MONTH"] = df["DATETIME"].dt.month
    df["YEAR"] = df["DATETIME"].dt.year
    df["ISO_YEAR"] = df["DATETIME"].dt.isocalendar().year.astype('int')
    df["QUARTER"] = df["DATETIME"].dt.quarter
    df["DAYOFYEAR"] = df["DATETIME"].dt.dayofyear
    df["DAYOFMONTH"] = df["DATETIME"].dt.day
    df["ISO_WEEKOFYEAR"] = df["DATETIME"].dt.isocalendar().week.astype('int')
    df["SEASON"] = df["DATETIME"].dt.month%12 // 3 + 1 # Season 1 is Summer
    return df

create_new_features_td(df_td_vis)

,DATETIME,TOTALDEMAND,REGIONID,HOUR,ISO_DAYOFWEEK,MONTH,YEAR,ISO_YEAR,QUARTER,DAYOFYEAR,DAYOFMONTH,ISO_WEEKOFYEAR,SEASON
0,2010-01-01 00:00:00,5561.21,QLD,0,5,1,2010,2009,1,1,1,53,1
1,2010-01-01 00:30:00,5422.25,QLD,0,5,1,2010,2009,1,1,1,53,1
2,2010-01-01 01:00:00,5315.98,QLD,1,5,1,2010,2009,1,1,1,53,1
3,2010-01-01 01:30:00,5186.70,QLD,1,5,1,2010,2009,1,1,1,53,1
4,2010-01-01 02:00:00,5050.83,QLD,2,5,1,2010,2009,1,1,1,53,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
786046,2021-03-17 22:00:00,7419.77,NSW,22,3,3,2021,2021,1,76,17,11,2
786047,2021-03-17 22:30:00,7417.91,NSW,22,3,3,2021,2021,1,76,17,11,2
786048,2021-03-17 23:00:00,7287.32,NSW,23,3,3,2021,2021,1,76,17,11,2
786049,2021-03-17 23:30:00,7172.39,NSW,23,3,3,2021,2021,1,76,17,11,2


In [ ]:
#df_td_vis.to_csv('R_Visualisation_v2/td_fourStates.csv', index=False)

<div style="background-color:rgba(10,219,0,0.5);font-size:24px;padding-left:20px">Temperature</div>

In [ ]:
# Merge all Total Demand datasets together into one
df_temp = pd.concat([df_temp_qld, df_temp_sa, df_temp_vic, df_temp_nsw]).reset_index(drop=True)

In [ ]:
df_temp.head(5)

,LOCATION,DATETIME,TEMPERATURE,Unnamed: 0
0,Brisbane Archerfield Airport,1/01/2010 0:00,23.6,NaN
1,Brisbane Archerfield Airport,1/01/2010 0:30,23.7,NaN
2,Brisbane Archerfield Airport,1/01/2010 1:00,23.5,NaN
3,Brisbane Archerfield Airport,1/01/2010 1:30,22.6,NaN
4,Brisbane Archerfield Airport,1/01/2010 2:00,22.0,NaN


In [ ]:
# Removing Unnamed: 0 column
df_temp.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778177 entries, 0 to 778176
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   LOCATION     778177 non-null  object 
 1   DATETIME     778177 non-null  object 
 2   TEMPERATURE  778177 non-null  float64
dtypes: float64(1), object(2)
memory usage: 17.8+ MB


In [ ]:
df_temp.describe()

,TEMPERATURE
count,778177.000000
mean,18.689432
std,5.980095
min,-1.300000
25%,14.300000
50%,18.900000
75%,22.900000
max,44.700000


In [ ]:
# Checking missing values 
missing_values(df_temp)

'No Missing Values'

### LOCATION Column

In [ ]:
# Let's get the unique values
df_temp['LOCATION'].unique()

array(['Brisbane Archerfield Airport', 'Adelaide (Kent Town)',
       'Melbourne (Olympic Park)', 'Bankstown'], dtype=object)

In [ ]:
# Making the assumption that the LOCATION represents REGIONID
df_temp.replace({'Brisbane Archerfield Airport': 'QLD', 
                 'Adelaide (Kent Town)': 'SA', 
                 'Melbourne (Olympic Park)': 'VIC',
                 'Bankstown': 'NSW'}, inplace=True)

# Changing column name to REGIONID to match other datasets
df_temp.rename({'LOCATION': 'REGIONID'}, axis=1, inplace=True)

In [ ]:
# Check to make sure there correct
unique_regionsTd = list(df_temp['REGIONID'].unique())
unique_regionsTd

['QLD', 'SA', 'VIC', 'NSW']

### DATETIME Column

In [ ]:
### For each LOCATION, let's make sure DATETIMES are in 'yyyy-mm-dd hh:mm:ss' format
for i in df_temp.groupby('REGIONID'):
    print(i)

('NSW',        REGIONID         DATETIME  TEMPERATURE
557851      NSW    1/1/2010 0:00         23.1
557852      NSW    1/1/2010 0:01         23.1
557853      NSW    1/1/2010 0:30         22.9
557854      NSW    1/1/2010 0:50         22.7
557855      NSW    1/1/2010 1:00         22.6
...         ...              ...          ...
778172      NSW  17/3/2021 23:00         19.1
778173      NSW  17/3/2021 23:20         19.0
778174      NSW  17/3/2021 23:30         18.8
778175      NSW  17/3/2021 23:34         18.8
778176      NSW   18/3/2021 0:00         18.6

[220326 rows x 3 columns])
('QLD',        REGIONID          DATETIME  TEMPERATURE
0           QLD    1/01/2010 0:00         23.6
1           QLD    1/01/2010 0:30         23.7
2           QLD    1/01/2010 1:00         23.5
3           QLD    1/01/2010 1:30         22.6
4           QLD    1/01/2010 2:00         22.0
...         ...               ...          ...
208080      QLD  17/03/2021 22:00         19.6
208081      QLD  17/03/2021 

In [ ]:
# Only ADELAIDE is in 'yyyy-mm-dd hh:mm:ss' format, so let's fix it
for region in unique_regionsTd:
    if region in ['QLD', 'VIC', 'NSW']:
        df_temp.loc[df_temp['REGIONID'] == region, 'DATETIME'] = pd.to_datetime(df_temp.loc[df_temp['REGIONID'] == region, 'DATETIME'], format='%d/%m/%Y %H:%M')
    else:
        df_temp.loc[df_temp['REGIONID'] == region, 'DATETIME'] = pd.to_datetime(df_temp.loc[df_temp['REGIONID'] == region, 'DATETIME'], format='%Y-%m-%d %H:%M:%S')
        
# Now converting 'DATETIME' column to datetime64[ns]
df_temp['DATETIME'] = pd.to_datetime(df_temp['DATETIME'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_temp

,REGIONID,DATETIME,TEMPERATURE
0,QLD,2010-01-01 00:00:00,23.6
1,QLD,2010-01-01 00:30:00,23.7
2,QLD,2010-01-01 01:00:00,23.5
3,QLD,2010-01-01 01:30:00,22.6
4,QLD,2010-01-01 02:00:00,22.0
...,...,...,...
778172,NSW,2021-03-17 23:00:00,19.1
778173,NSW,2021-03-17 23:20:00,19.0
778174,NSW,2021-03-17 23:30:00,18.8
778175,NSW,2021-03-17 23:34:00,18.8


In [ ]:
# Making a copy of the data for experimentation
df_temp2 = df_temp.copy()

# Creating 3 new variables
df_temp2['yyyy'] = df_temp2['DATETIME'].dt.year
df_temp2['mm'] = df_temp2['DATETIME'].dt.month
df_temp2['dd'] = df_temp2['DATETIME'].dt.day

In [ ]:
df_temp2.head(5)

,REGIONID,DATETIME,TEMPERATURE,yyyy,mm,dd
0,QLD,2010-01-01 00:00:00,23.6,2010,1,1
1,QLD,2010-01-01 00:30:00,23.7,2010,1,1
2,QLD,2010-01-01 01:00:00,23.5,2010,1,1
3,QLD,2010-01-01 01:30:00,22.6,2010,1,1
4,QLD,2010-01-01 02:00:00,22.0,2010,1,1


In [ ]:
days_in_year = {2010: 365, 2011: 365, 2012: 366, 2013: 365, 2014: 365, 2015: 365,
                2016: 366, 2017: 365, 2018: 365, 2019: 365, 2020: 366, 2021: 365}

days_in_month = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
                 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

In [ ]:
# Checks whether there is any missing days in the data
def check_for_days_in_month(df, col='REGIONID', regions=unique_regionsTd):
    for state in regions:
        df_daysInMonthPerYear = df[df[col] == state].groupby(['yyyy', 'mm'])[['dd']].nunique().reset_index()
        df_daysInYear = df[df[col] == state].groupby(['yyyy', 'mm'])[['dd']].nunique().reset_index()[['yyyy', 'dd']].groupby('yyyy')[['dd']].agg('sum').reset_index()
        
        for _, row in df_daysInMonthPerYear.iterrows():
            yyyy, mm, dd = row['yyyy'], row['mm'], row['dd']
            
            daysInYear = days_in_year[yyyy]
            
            # If leap year
            if yyyy in [2012, 2016, 2020] and mm == 2:
                daysInMonth = days_in_month[mm] + 1
            else:
                daysInMonth = days_in_month[mm]

            if dd == daysInMonth:
                pass
            else:
                print(f'{state}: There are {daysInMonth - dd} missing days in year {yyyy} and month {mm}')
        print('')

check_for_days_in_month(df_temp2)

QLD: There are 7 missing days in year 2016 and month 8
QLD: There are 13 missing days in year 2021 and month 3

SA: There are 7 missing days in year 2016 and month 8
SA: There are 13 missing days in year 2021 and month 3

VIC: There are 30 missing days in year 2013 and month 5
VIC: There are 13 missing days in year 2021 and month 3

NSW: There are 3 missing days in year 2016 and month 7
NSW: There are 13 missing days in year 2021 and month 3



There are missing days in QLD in 2016-08, SA in 2016-08, and NSW 2021-07. The others are just the start and end months of when the time series begun and finished respectively.

In [ ]:
# Lets see what days are missing for all 4 cities on August 2016
# Lets check what days are missing in August, 2016
def return_missing_days(df, col, region, yyyy, mm, days=31):
    missing_days = np.sort(
        df[(df['yyyy'] == yyyy) & (df['mm'] == mm)]
        .loc[df[col] == region, 'dd']
        .unique()
    )
    missing_days = list(set(range(1,days+1)) - set(missing_days))

    if len(missing_days) == 0:
        pass
    else:
        print(f'The missing days in {region} in year {yyyy} and month {mm} are: {missing_days}')

return_missing_days(df_temp2, col='REGIONID', region='QLD', yyyy=2016, mm=8)
return_missing_days(df_temp2, col='REGIONID', region='SA', yyyy=2016, mm=8)
return_missing_days(df_temp2, col='REGIONID', region='NSW', yyyy=2016, mm=7)
# NOTES:
### (1) when we perform the .resample() method the below missing days/datetimes will be added inducing NaNs for their values
### (2) all locations have 13 missing days in year 2021 and month 3 as the final DATETIME is '2021-03-18 00:00:00'
### (3) The DATETIME for Melbourne starts at '2013-05-31 15:00:00' as to why it has 30 missing days in year 2013 and month 5

The missing days in QLD in year 2016 and month 8 are: [24, 25, 26, 27, 28, 29, 30]
The missing days in SA in year 2016 and month 8 are: [24, 25, 26, 27, 28, 29, 30]
The missing days in NSW in year 2016 and month 7 are: [16, 17, 18]


In [ ]:
df_temp3 = df_temp2.copy()

In [ ]:
df_temp3.head()

,REGIONID,DATETIME,TEMPERATURE,yyyy,mm,dd
0,QLD,2010-01-01 00:00:00,23.6,2010,1,1
1,QLD,2010-01-01 00:30:00,23.7,2010,1,1
2,QLD,2010-01-01 01:00:00,23.5,2010,1,1
3,QLD,2010-01-01 01:30:00,22.6,2010,1,1
4,QLD,2010-01-01 02:00:00,22.0,2010,1,1


In [ ]:
# The DATETIMES arn't in half-hourly intervals, so we apply the .resample() method
def apply_resample(df, col, regions=unique_regionsTd):
    empty_df = pd.DataFrame()
    
    for region in regions:
        df_region = df.loc[df[col] == region, :]
        df_region = df_region.resample('30T', on='DATETIME', closed='left').mean().reset_index(drop=False).assign(REGIONID=f'{region}')
        empty_df = empty_df.append(df_region, ignore_index=True, sort=False)

    # Fixing the yyyy, mm, and dd columns
    empty_df['yyyy'] = empty_df['DATETIME'].dt.year
    empty_df['mm'] = empty_df['DATETIME'].dt.month
    empty_df['dd'] = empty_df['DATETIME'].dt.day
    
    return empty_df.reset_index(drop=True)
    
df_temp3 = apply_resample(df_temp3, col='REGIONID')

In [ ]:
df_temp3

,DATETIME,TEMPERATURE,yyyy,mm,dd,REGIONID
0,2010-01-01 00:00:00,23.60,2010,1,1,QLD
1,2010-01-01 00:30:00,23.70,2010,1,1,QLD
2,2010-01-01 01:00:00,23.50,2010,1,1,QLD
3,2010-01-01 01:30:00,22.60,2010,1,1,QLD
4,2010-01-01 02:00:00,22.00,2010,1,1,QLD
...,...,...,...,...,...,...
726209,2021-03-17 22:00:00,19.70,2021,3,17,NSW
726210,2021-03-17 22:30:00,19.50,2021,3,17,NSW
726211,2021-03-17 23:00:00,19.05,2021,3,17,NSW
726212,2021-03-17 23:30:00,18.80,2021,3,17,NSW


In [ ]:
# Check for any duplicate DATETIMES
check_duplicate_datetimes(df_temp3)

QLD:
Column = 196513   |   Column Uniques = 196513   |   All Combinations = 196513

SA:
Column = 196513   |   Column Uniques = 196513   |   All Combinations = 196513

VIC:
Column = 136675   |   Column Uniques = 136675   |   All Combinations = 136675

NSW:
Column = 196513   |   Column Uniques = 196513   |   All Combinations = 196513



In [ ]:
# After applying the .resample() method all DATETIMES for each location between min and max are there
check_missing_datetimes(df_temp3, REGIONIDS=unique_regionsTd, col='REGIONID')

REGIONID QLD has no missing DATETIME between 2010-01-01 00:00:00 and 2021-03-18 00:00:00
REGIONID SA has no missing DATETIME between 2010-01-01 00:00:00 and 2021-03-18 00:00:00
REGIONID VIC has no missing DATETIME between 2013-05-31 15:00:00 and 2021-03-18 00:00:00
REGIONID NSW has no missing DATETIME between 2010-01-01 00:00:00 and 2021-03-18 00:00:00


In [ ]:
# Checking for missing values in each group
for region, group in df_temp3.groupby('REGIONID'):
    print(region, '\n', missing_values(group), '\n\n')

NSW 
              Total  Percentage
TEMPERATURE    559        0.28 


QLD 
              Total  Percentage
TEMPERATURE    779         0.4 


SA 
              Total  Percentage
TEMPERATURE    779         0.4 


VIC 
              Total  Percentage
TEMPERATURE     67        0.05 




In [ ]:
# Let's determine how many consecutive missing half-hour intervals there are for each location
# Let's see how many consecutive missing half-hourly temperature values there are 
def consecutive_missings(df, col, regions):
    first_nan = True
    count_consecutive_NaN = 0
    tracking = {'start_datetime': [], 'end_datetime': [], 'consecutive_NaN': [], 'region': []}
    
    for region in regions:
        df1 = df.loc[df[col] == region, :]
        
        for i, row in df1.iterrows():
            datetime, temp, region = row['DATETIME'], row['TEMPERATURE'], row['REGIONID']
            if pd.isnull(temp):
                count_consecutive_NaN += 1
                if first_nan is True:
                    tracking['start_datetime'].append(datetime)
                    tracking['region'].append(region)
                    first_nan = False
            else:
                if first_nan is False:
                    datetime = datetime + pd.Timedelta(minutes=-30)
                    tracking['end_datetime'].append(datetime)
                    tracking['consecutive_NaN'].append(count_consecutive_NaN)
                    first_nan = True
                count_consecutive_NaN = 0
            
    return pd.DataFrame(tracking)

consec_missing = consecutive_missings(df_temp3, col='REGIONID', regions=unique_regionsTd)

In [ ]:
consec_missing
# Split the dataset into 2 based on the number of consecutive_NaN.

In [ ]:
# Let's split the 'consec_missing' dataframe into 2 dataframes based on number of consecutive missing days
consec_missing_lt10 = consec_missing[consec_missing['consecutive_NaN'] < 10]
consec_missing_gte10 = consec_missing[consec_missing['consecutive_NaN'] >= 10]

In [ ]:
# Let's create that a function that will remove all dates (yyyy-mm-dd) that contain X or more consecutive missing values in a row, and linear intepolate if less than X
def fix_temp_missing_values(df, consec_mvs1, consec_mvs2):
    # Interpolation Loop
    for i, row in consec_mvs1.iterrows():
        start, end, running_NaN, region = row['start_datetime'], row['end_datetime'], row['consecutive_NaN'], row['region']
        start_less30 = start + pd.Timedelta(minutes=-30)
        end_plus30 = end + pd.Timedelta(minutes=30)

        # Linear interpolate temperatures with less than 10 consecutive missing values
        date_range = list(pd.date_range(start=start_less30, end=end_plus30, freq='30T'))
        df.loc[(df['DATETIME'].isin(date_range)) & (df['REGIONID'] == region), 'TEMPERATURE'] = df.loc[df['DATETIME'].isin(date_range), 'TEMPERATURE'].interpolate()

    # Removal of Days
    for i, row in consec_mvs2.iterrows():
        start, end, running_NaN, region = row['start_datetime'], row['end_datetime'], row['consecutive_NaN'], row['region']
        start_less30 = start + pd.Timedelta(minutes=-30)
        end_plus30 = end + pd.Timedelta(minutes=30)
        
        start_date = dt.datetime.strftime(start, '%Y-%m-%d')
        end_date = dt.datetime.strftime(end, '%Y-%m-%d')
        end_datePlus1 = dt.datetime.strftime(end + pd.Timedelta(days=1), '%Y-%m-%d')

        if start_date == end_date:
            date_range2 = list(pd.date_range(start=start_date, periods=48, closed='left', freq='30T'))
        elif start_date != end_date:
            date_range2 = list(pd.date_range(start=start_date, end=end_datePlus1, closed='left', freq='30T'))

        rows_to_drop = df.loc[(df['DATETIME'].isin(date_range2)) & (df['REGIONID'] == region), :].index
        df.drop(rows_to_drop, axis=0, inplace=True)

    return df
 
df_temp4 = fix_temp_missing_values(df_temp3, consec_missing_lt10, consec_missing_gte10).reset_index(drop=True)

In [ ]:
# Check for no missing values
missing_values(df_temp4)

'No Missing Values'

### TEMPERATURE Column

See descriptive statistics above for TEMPERATURE column.

## Visualisation

The data has been cleaned into a better format, let's now visualise the data.

In [ ]:
# Create a copy of the dataset for visualisation
df_temp_vis = df_temp4.copy()

# Remove yyyy, mm, dd columns (these will be recomputed)
df_temp_vis.drop(['yyyy', 'mm', 'dd'], axis=1, inplace=True)

In [ ]:
# First, we'll introduce some new variables based on the DATETIME column to make visualisation easier
# Let's create some new features derived from DATETIME
df_temp_vis = create_new_features_td(df_temp_vis)
df_temp_vis

,DATETIME,TEMPERATURE,REGIONID,HOUR,ISO_DAYOFWEEK,MONTH,YEAR,ISO_YEAR,QUARTER,DAYOFYEAR,DAYOFMONTH,ISO_WEEKOFYEAR,SEASON
0,2010-01-01 00:00:00,23.60,QLD,0,5,1,2010,2009,1,1,1,53,1
1,2010-01-01 00:30:00,23.70,QLD,0,5,1,2010,2009,1,1,1,53,1
2,2010-01-01 01:00:00,23.50,QLD,1,5,1,2010,2009,1,1,1,53,1
3,2010-01-01 01:30:00,22.60,QLD,1,5,1,2010,2009,1,1,1,53,1
4,2010-01-01 02:00:00,22.00,QLD,2,5,1,2010,2009,1,1,1,53,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
723809,2021-03-17 22:00:00,19.70,NSW,22,3,3,2021,2021,1,76,17,11,2
723810,2021-03-17 22:30:00,19.50,NSW,22,3,3,2021,2021,1,76,17,11,2
723811,2021-03-17 23:00:00,19.05,NSW,23,3,3,2021,2021,1,76,17,11,2
723812,2021-03-17 23:30:00,18.80,NSW,23,3,3,2021,2021,1,76,17,11,2


In [ ]:
#df_temp_vis.to_csv('R_Visualisation_v2/temp_fourStates.csv', index=False)

<div style="background-color:rgba(10,219,0,0.5);font-size:24px;padding-left:20px">Forecast Demand</div>

In [ ]:
# Merge all Total Demand datasets together into one
df_fc = pd.concat([df_fc_qld, df_fc_sa, df_fc_vic, df_fc_nsw]).reset_index(drop=True)

In [ ]:
df_fc.head(5)

,PREDISPATCHSEQNO,REGIONID,PERIODID,FORECASTDEMAND,LASTCHANGED,DATETIME
0,2016123018,QLD1,71,6425.36,2016-12-30 12:31:25,2017-01-01 00:00:00
1,2016123018,QLD1,72,6193.34,2016-12-30 12:31:25,2017-01-01 00:30:00
2,2016123018,QLD1,73,6016.45,2016-12-30 12:31:25,2017-01-01 01:00:00
3,2016123018,QLD1,74,5859.23,2016-12-30 12:31:25,2017-01-01 01:30:00
4,2016123018,QLD1,75,5767.15,2016-12-30 12:31:25,2017-01-01 02:00:00


In [ ]:
df_fc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23192795 entries, 0 to 23192794
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   PREDISPATCHSEQNO  int64  
 1   REGIONID          object 
 2   PERIODID          int64  
 3   FORECASTDEMAND    float64
 4   LASTCHANGED       object 
 5   DATETIME          object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.0+ GB


In [ ]:
df_fc.describe()

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND
count,2.319280e+07,2.319280e+07,2.319280e+07
mean,2.017036e+09,2.998145e+01,5.979442e+03
std,2.962086e+06,1.869739e+01,2.693715e+03
min,2.009123e+09,1.000000e+00,1.198300e+02
25%,2.015122e+09,1.400000e+01,4.542080e+03
50%,2.018033e+09,2.800000e+01,6.393090e+03
75%,2.019092e+09,4.400000e+01,7.958050e+03
max,2.021032e+09,7.900000e+01,1.473666e+04


In [ ]:
# Checking missing values 
missing_values(df_fc)

'No Missing Values'

### REGIONID Column

In [ ]:
# Let's get the unique values
df_fc['REGIONID'].unique()

array(['QLD1', 'SA1', 'VIC1', 'NSW1'], dtype=object)

In [ ]:
# Removing the 1 on the end of each value
df_fc.replace({'QLD1': 'QLD', 'SA1': 'SA', 'VIC1': 'VIC', 'NSW1': 'NSW'}, inplace=True)

In [ ]:
# Check to make sure there correct
unique_regions_fc = list(df_fc['REGIONID'].unique())
unique_regions_fc

['QLD', 'SA', 'VIC', 'NSW']

### PREDISPATCHSEQNO Column

In [ ]:
# Let's create a copy of the dataset
df_fc2 = df_fc.copy()

# Let's convert PREDISPATCHSEQNO column to string to perform slicing
df_fc2['PREDISPATCHSEQNO'] = df_fc2['PREDISPATCHSEQNO'].astype('str')

In [ ]:
#############################################################################################
####### Lets do some checking of the values to make sure there are no inconsistencies #######
#############################################################################################
def check_uniques(df, col='PREDISPATCHSEQNO'):
    # Prints out the PREDISPATCHSEQNO (yyyymmddpp) that are not 10 characters (digits) long (if any)
    for x in df['PREDISPATCHSEQNO']:
        if len(x) != 10:
            print(x)

    # Lets check the unique years, months, days, and pp values 
    years, months, days, pps = [], [], [], []
    for x in df['PREDISPATCHSEQNO']:
        year, month, day, pp = x[0:4], x[4:6], x[6:8], x[8:]
        
        years.append(year)
        months.append(month)
        days.append(day)
        pps.append(pp)
        
    years = sorted(list(set(years)))    # 2016-2021
    months = sorted(list(set(months)))  # 01-12
    days = sorted(list(set(days)))      # 01-31
    pps = sorted(list(set(pps)))        # 01-48
        
    print(f'Unique years:\n{years}\n\n'
          f'Unique months:\n{months}\n\n'
          f'Unique days:\n{days}\n\n'
          f'Unique pps:\n{pps}\n')

In [ ]:
for region, group in df_fc2.groupby('REGIONID'):
    unique_pre = pd.DataFrame(group['PREDISPATCHSEQNO'].unique(), columns=['PREDISPATCHSEQNO'])
    print(region, '\n')
    check_uniques(unique_pre)

NSW 

Unique years:
['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

Unique months:
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

Unique days:
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']

Unique pps:
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48']

QLD 

Unique years:
['2016', '2017', '2018', '2019', '2020', '2021']

Unique months:
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

Unique days:
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '

All regions have the same months, days and pp values, however, NSW dates back to 2009, and not 2016, for years.

In [ ]:
# Each pp on the end of PREDISPATCHSEQNO (yyyymmddpp) must have the following counts, if not then there may be error
# pp_mappings = {'01': 48, '02': 47, '03': 46, '04': 45, '05': 44, '06': 43, '07': 42, '08': 41, '09': 40, '10': 39,
#                '11': 38, '12': 37, '13': 36, '14': 35, '15': 34, '16': 33, '17': 32, '18': 79, '19': 78, '20': 77,
#                '21': 76, '22': 75, '23': 74, '24': 73, '25': 72, '26': 71, '27': 70, '28': 69, '29': 68, '30': 67,
#                '31': 66, '32': 65, '33': 64, '34': 63, '35': 62, '36': 61, '37': 60, '38': 59, '39': 58, '40': 57,
#                '41': 56, '42': 55, '43': 54, '44': 53, '45': 52, '46': 51, '47': 50, '48': 49}

# def check_pp_counts(df, groupByCol='PREDISPATCHSEQNO', col='PERIODID', pp_mappings=pp_mappings):
#     for PREDISPATCHSEQNO, PERIODID in df.groupby(groupByCol)[col]:
#         pp = str(PREDISPATCHSEQNO)[8:]
#         group_count = PERIODID.agg('count')
#         if pp_mappings[pp] != group_count:
#             print(f'PREDISPATCHSEQNO: {PREDISPATCHSEQNO}     |     Counts: {group_count}') 

# for region, group in df_fc2.groupby('REGIONID'):
#     print(f'{region}\n')
#     check_pp_counts(group)
#     print(f'\n\n')

In [ ]:
# For each unique yyyymmdd (not including pp) in the PREDISPATCHSEQNO there should be pp values of 01-48 (i.e. 48 PREDISPATCHSEQNO per unique date yyyymmdd)
# Lets check that there is 48 pp values (01-48) associated with each unique date (yyyymmdd) in the PREDISPATCHSEQNO
def check_date_has_48_pp(df, col='PREDISPATCHSEQNO'):
    # Define dict to map yyyymmdd to number of associated pp values
    counts = dict()
    
    for yyyymmddpp in df[col]:
        yyyymmdd = yyyymmddpp[0:8]
        pp = yyyymmddpp[8:]
        
        if yyyymmdd not in counts:
            counts[yyyymmdd] = 1
        else:
            counts[yyyymmdd] += 1
            
    # Sort dictionary by key in ascending order        
    counts = {k:v for k,v in sorted(counts.items(), key=lambda x: x[0])}
    
    for k, v in counts.items():
        if v != 48:
            print(f'Date: {k[0:4]}-{k[4:6]}-{k[6:8]}    |    Counted: {v}    |    Not-Counted: {48 - int(v)}')

In [ ]:
for region, group in df_fc2.groupby('REGIONID'):
    unique_pre = pd.DataFrame(group['PREDISPATCHSEQNO'].unique(), columns=['PREDISPATCHSEQNO'])
    print(region, '\n')
    check_date_has_48_pp(unique_pre)
    print('\n\n')

NSW 

Date: 2009-12-30    |    Counted: 31    |    Not-Counted: 17
Date: 2015-05-11    |    Counted: 47    |    Not-Counted: 1
Date: 2017-01-20    |    Counted: 47    |    Not-Counted: 1
Date: 2017-10-11    |    Counted: 47    |    Not-Counted: 1
Date: 2017-12-21    |    Counted: 47    |    Not-Counted: 1
Date: 2018-04-10    |    Counted: 47    |    Not-Counted: 1
Date: 2018-05-03    |    Counted: 47    |    Not-Counted: 1
Date: 2018-05-07    |    Counted: 47    |    Not-Counted: 1
Date: 2018-05-10    |    Counted: 47    |    Not-Counted: 1
Date: 2018-05-26    |    Counted: 47    |    Not-Counted: 1
Date: 2018-11-22    |    Counted: 47    |    Not-Counted: 1
Date: 2019-10-10    |    Counted: 47    |    Not-Counted: 1
Date: 2020-10-03    |    Counted: 47    |    Not-Counted: 1
Date: 2021-03-17    |    Counted: 40    |    Not-Counted: 8



QLD 

Date: 2016-12-30    |    Counted: 31    |    Not-Counted: 17
Date: 2017-01-20    |    Counted: 47    |    Not-Counted: 1
Date: 2017-10-11    |  

There seems to be a few missing PREDISPATCHSEQNO for all 4 states. Let's determine precisely what dates these were. 

In [ ]:
# Lets find the missing pp values associated with the above dates for all 4 locations
pp_suffix = ['01', '02', '03', '04', '05', '06', '07' ,'08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
             '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
             '41', '42', '43', '44', '45', '46', '47', '48']

def find_missing_PREDISPATCHSEQNO(df, col='PREDISPATCHSEQNO', pp_suffix=pp_suffix):
    every_PREDISPATCHSEQNO_comb = []
    every_PREDISPATCHSEQNO_without_pp = df.apply(lambda x: x[col][0:8], axis=1).unique()
    min_, max_ = np.min(every_PREDISPATCHSEQNO_without_pp), np.max(every_PREDISPATCHSEQNO_without_pp)
    
    for yyyymmdd in every_PREDISPATCHSEQNO_without_pp:
        if yyyymmdd == min_:
            for pp in pp_suffix[17:]:
                yyyymmddpp = yyyymmdd + pp
                every_PREDISPATCHSEQNO_comb.append(yyyymmddpp)
        elif yyyymmdd == max_:
            for pp in pp_suffix[:40]:
                yyyymmddpp = yyyymmdd + pp
                every_PREDISPATCHSEQNO_comb.append(yyyymmddpp)
        else:
            for pp in pp_suffix:
                yyyymmddpp = yyyymmdd + pp
                every_PREDISPATCHSEQNO_comb.append(yyyymmddpp)
            
    every_missing_PREDISPATCHSEQNO = sorted(list(set(every_PREDISPATCHSEQNO_comb) - set(df[col])))
    return every_missing_PREDISPATCHSEQNO

In [ ]:
missing_PREDISPATCHSEQNO_per_location = dict()

for region, group in df_fc2.groupby('REGIONID'):
    unique_pre = pd.DataFrame(group['PREDISPATCHSEQNO'].unique(), columns=['PREDISPATCHSEQNO'])
    missings = find_missing_PREDISPATCHSEQNO(unique_pre)
    missing_PREDISPATCHSEQNO_per_location[region] = missings

In [ ]:
# All the missing PREDISPATCHSEQNO for each REGIONID between min and max PREDISPATCHSEQNO for each location
missing_PREDISPATCHSEQNO_per_location

{'NSW': ['2015051116',
  '2017012036',
  '2017101138',
  '2017122120',
  '2018041020',
  '2018050328',
  '2018050732',
  '2018051019',
  '2018052610',
  '2018112224',
  '2019101001',
  '2020100324'],
 'QLD': ['2017012036',
  '2017101138',
  '2017122120',
  '2018041020',
  '2018050328',
  '2018050732',
  '2018051019',
  '2018052610',
  '2018112224',
  '2019101001',
  '2020100324'],
 'SA': ['2017012036',
  '2017101138',
  '2017122120',
  '2018041020',
  '2018050328',
  '2018050732',
  '2018051019',
  '2018052610',
  '2018112224',
  '2019101001',
  '2020100324'],
 'VIC': ['2017012036',
  '2017101138',
  '2017122120',
  '2018041020',
  '2018050328',
  '2018050732',
  '2018051019',
  '2018052610',
  '2018112224',
  '2019101001',
  '2020100324']}

### DATETIME column

In [ ]:
df_fc3 = df_fc2.copy()

In [ ]:
# Convert LASTCHANGED and DATETIME columns to datetime64[ns]
df_fc3['DATETIME'] = pd.to_datetime(df_fc3['DATETIME'])
df_fc3['LASTCHANGED'] = pd.to_datetime(df_fc3['LASTCHANGED'])

# Adding HOUR and MINUTE columns
df_fc3['HOUR'] = df_fc3['DATETIME'].dt.hour
df_fc3['MINUTE'] = df_fc3['DATETIME'].dt.minute

In [ ]:
# Checking hour is number between 0 and 23
df_fc3['HOUR'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [ ]:
# Checking minute is either 0 or 30
df_fc3['MINUTE'].unique()

array([ 0, 30])

In [ ]:
df_fc3

,PREDISPATCHSEQNO,REGIONID,PERIODID,FORECASTDEMAND,LASTCHANGED,DATETIME,HOUR,MINUTE
0,2016123018,QLD,71,6425.36,2016-12-30 12:31:25,2017-01-01 00:00:00,0,0
1,2016123018,QLD,72,6193.34,2016-12-30 12:31:25,2017-01-01 00:30:00,0,30
2,2016123018,QLD,73,6016.45,2016-12-30 12:31:25,2017-01-01 01:00:00,1,0
3,2016123018,QLD,74,5859.23,2016-12-30 12:31:25,2017-01-01 01:30:00,1,30
4,2016123018,QLD,75,5767.15,2016-12-30 12:31:25,2017-01-01 02:00:00,2,0
...,...,...,...,...,...,...,...,...
23192790,2021031736,NSW,5,7028.45,2021-03-17 21:31:31,2021-03-18 00:00:00,0,0
23192791,2021031737,NSW,4,7041.65,2021-03-17 22:01:34,2021-03-18 00:00:00,0,0
23192792,2021031738,NSW,3,7087.60,2021-03-17 22:31:36,2021-03-18 00:00:00,0,0
23192793,2021031739,NSW,2,7087.28,2021-03-17 23:01:36,2021-03-18 00:00:00,0,0


## Visualisation

The data has been analysed for any missing values, now we can proceed to visualise

In [ ]:
df_fc_vis = df_fc2.copy()

# Convert LASTCHANGED and DATETIME columns to datetime64[ns]
df_fc_vis['DATETIME'] = pd.to_datetime(df_fc_vis['DATETIME'], format='%Y-%m-%d %H:%M:%S')
df_fc_vis['LASTCHANGED'] = pd.to_datetime(df_fc_vis['LASTCHANGED'], format='%Y-%m-%d %H:%M:%S')

# Make 4 new columns based on PREDISPATCHSEQNO, suffixed with _pre
df_fc_vis['yyyy_pre'] = df_fc_vis['PREDISPATCHSEQNO'].apply(lambda x: x[0:4]).astype('int16')
df_fc_vis['mm_pre'] = df_fc_vis['PREDISPATCHSEQNO'].apply(lambda x: x[4:6]).astype('int16')
df_fc_vis['dd_pre'] = df_fc_vis['PREDISPATCHSEQNO'].apply(lambda x: x[6:8]).astype('int16')
df_fc_vis['pp_pre'] = df_fc_vis['PREDISPATCHSEQNO'].apply(lambda x: x[8:]).astype('int16')

In [ ]:
# Adding new features to visualise the data
df_fc_vis = create_new_features_td(df_fc_vis)
df_fc_vis

In [ ]:
# Convert int data types to int16 to save space
df_fc_vis = df_fc_vis.astype({col: 'int16' for col in df_fc_vis.select_dtypes('int64').columns})

In [ ]:
# Separating datasets in four based on REGIONID due to size.
# df_fc_vis[df_fc_vis['REGIONID'] == 'QLD'].to_csv('R_Visualisation_v2/fc_qld.csv', index=False)
# df_fc_vis[df_fc_vis['REGIONID'] == 'SA'].to_csv('R_Visualisation_v2/fc_sa.csv', index=False)
# df_fc_vis[df_fc_vis['REGIONID'] == 'VIC'].to_csv('R_Visualisation_v2/fc_vic.csv', index=False)
# df_fc_vis[df_fc_vis['REGIONID'] == 'NSW'].to_csv('R_Visualisation_v2/fc_nsw.csv', index=False)

<div style="background-color:rgba(10,219,0,0.5);font-size:24px;padding-left:20px">Merging Temperature and Total Demand</div>

In [ ]:
# Create copies of Temperature and Total Demand datasets first
df_temp_merge = df_temp4.copy()
df_td_merge = df_td.copy()

# Let's first drop yyyy, mm, dd in df_temp_merge
df_temp_merge.drop(['yyyy', 'mm', 'dd'], axis=1, inplace=True)

In [ ]:
df_tempTd_merged = df_temp_merge.merge(df_td_merge, how='inner', on=['DATETIME', 'REGIONID'])

In [ ]:
df_tempTd_merged

,DATETIME,TEMPERATURE,REGIONID,TOTALDEMAND
0,2010-01-01 00:00:00,23.60,QLD,5561.21
1,2010-01-01 00:30:00,23.70,QLD,5422.25
2,2010-01-01 01:00:00,23.50,QLD,5315.98
3,2010-01-01 01:30:00,22.60,QLD,5186.70
4,2010-01-01 02:00:00,22.00,QLD,5050.83
...,...,...,...,...
723808,2021-03-17 22:00:00,19.70,NSW,7419.77
723809,2021-03-17 22:30:00,19.50,NSW,7417.91
723810,2021-03-17 23:00:00,19.05,NSW,7287.32
723811,2021-03-17 23:30:00,18.80,NSW,7172.39


In [ ]:
# df_tempTd_merged.to_csv('R_Visualisation_v2/tempTd_fourStates.csv', index=False)